In [2]:
import json
import csv
import numpy as np

total_rows_added = 0

input_file_name = "/Users/jackgengqc/Desktop/yttemporal1b_train_0000of1024.jsonl"
output_csv_name = "./100_video_ids.csv"

# Open the input file for reading and the CSV file for writing
with open(input_file_name, 'r') as input_file, open(output_csv_name, 'w', newline='') as output_csv:
    csv_writer = csv.writer(output_csv)
    csv_writer.writerow(['id'])  # Write header

    # Process each line in the JSONL file
    for line in input_file:
        data = json.loads(line)

        subtitles_t_start = data["subtitles_t_start"]
        subtitles_t_end = data["subtitles_t_end"]
        duration = data["duration"]
        youtube_id = data['id']

        # Calculate time deltas between consecutive words
        time_deltas = [start - end for start, end in zip(subtitles_t_start[1:], subtitles_t_end[:-1])]

        # Calculate the mean of the time deltas
        mean_time_delta = np.mean(time_deltas)

        if mean_time_delta > 0.001665:
            continue

        if duration < 420:
            continue

        total_rows_added += 1

        # Write the qualified video ID to the CSV file
        csv_writer.writerow([youtube_id])

        if total_rows_added == 100:
            break

    print(f"Total rows added: {total_rows_added}")

print("File processed.")


Total rows added: 100
File processed.
